# Getting Started with Anovos

_Anovos_ provides data scientists and ML engineers with powerful and versatile tools for feature engineering.

In this guide, you will learn how to set up get to know key capabilities of _Anovos_.

## Setting up and verifying the Python and Spark environment

_Anovos_ builds on [Apache Spark](https://spark.apache.org/), a highly scalable engine for data engineering. Thus, an installation of Spark is required to run any _Anovos_ code. Further, the Python bindings for Spark (known as `pyspark`) need to be installed in a compatible version.

If you are first starting out with _Anovos_ and are not yet familiar with Spark, we recommend you execute this guide through the provided _anovos-demo_ Docker container, which provides a full Spark setup along with compatible versions of Python and a Jupyter notebook environment.

**TODO** Add link to how-to for installing Spark and Python

**TODO** Add link to Anovos documentation that details how Spark needs to be configured

The currently available Beta version of _Anovos_ is specifically built for Spark 2.4.x, Python 3.7.x, and Java 8 (OpenJDK 1.8.x). You can verify that you're running the correct versions by executing the following lines:

In [ ]:
!python --version

In [ ]:
!spark-submit --version

Let's also check that a compatible version of `pyspark` is available within our Python environment: 

In [ ]:
import pyspark
pyspark.__version__

If you haven't done so already, let's install _Anovos_ into the currently active Python environment:

In [ ]:
!pip install anovos

As _Anovos_ relies on Spark behind the scenes for most of the heavy lifting, we need to pass an instantiated [`SparkSession`](https://spark.apache.org/docs/2.4.8/api/python/pyspark.sql.html?highlight=sparksession#pyspark.sql.SparkSession) to many of the function calls.

**TODO** Add link to Anovos documentation that details how Spark needs to be configured

For the purposes of this guide, we'll use the pre-configured `SparkSession` instance provided by [`anovos.shared.spark`](https://github.com/anovos/anovos/blob/main/src/main/anovos/shared/spark.py):

In [ ]:
from anovos.shared.spark import spark

(Don't worry if this import takes some time and prints a lot of output. You should see that settings are loaded, dependencies are added, and the logger is configured.)

## Loading data

Data ingestion is the first step in any feature engineering project. _Anovos_ builds on [Spark's data loading capabilites](https://spark.apache.org/docs/2.4.8/api/python/pyspark.sql.html) and can handle different common file formats such as CSV, Parquet, and Avro.

_Anovos_' `data_ingest` module provides all data ingestion functionality. It includes functions to merge multiple datasets as well as to select subsets of the loaded data.

Let's load the classic [Adult Income dataset](https://archive.ics.uci.edu/ml/datasets/adult) in CSV format, which we'll use throughout this guide:

In [ ]:
from anovos.data_ingest.data_ingest import read_dataset

df = read_dataset(
    spark,  # Remember: The first argument of Anovos functions is always an instantiated SparkSession
    file_path='../data/income_dataset/csv',
    file_type='csv',
    file_configs={'header': 'True', 'delimiter': ',', 'inferSchema': 'True'}
)

Note that `df` is a standard Spark [`DataFrame`](https://spark.apache.org/docs/2.4.8/api/python/pyspark.sql.html?highlight=sparksession#pyspark.sql.DataFrame):

In [ ]:
type(df)

Thus, you can use all the built-in methods you might be familiar with, e.g.

In [ ]:
df.printSchema()

In this guide, we will work with just a few of the columns in the dataset:

In [ ]:
from anovos.data_ingest.data_ingest import select_column

df = select_column(df, list_of_cols=['age', 'education', 'education-num', 'occupation', 'hours-per-week', 'income'])

## Learning about the data

Before we can start to engineer features for, we need to understand the data at hand. For example, we need to verify that the data has sufficient quality or whether we'll have to deal with missing values.

_Anovos_' `data_analyzer` module provides three submodules for this purpose:

- The functions of `data_analyzer.quality_checker` allow us to detect and fix issues such as empty rows or duplicate entries
- `data_analyzer.stats_generator` offers functions to caculate various statistical properties
- Finally, `data_analyzer.association_evaluator` enables us to examine a dataset for correlations between columns

### Assess the data quality

Once a new dataset is loaded, its quality should be assessed. Does the dataset contain all the columns we expect? Do we have duplicate values? Did we ingest the expected number of unique data points? _Anovos_' `data_analyzer.quality_checker` module provides convenient functions to answer these questions.

For sake of time, in this guide, we'll assume that our dataset does not exhibit such problems. Instead, we'll move on to a more advanced stage of quality assessment and check for outliers in the data. Outliers are data points that deviate significantly from the others. These points can be problematic when training ML models or during inference, as there is very little information about these ranges in the dataset, which in turn leads to a high degree of uncertainty. 

Outliers in a dataset can be due to rare events. For example, in the case of our income dataset, there might be some individuals that work well beyond retirement age or have worked their way up to a high paying position despite dropping out of high school.

However, outliers can also be due to reporting errors. Digits in a number might have been swapped or values might have been given in the wrong order of magnitude. For example, someone might have mistakenly entered a weight in grams instead of kilograms.

How we deal with outliers depends on their origin and the application context.

In [ ]:
from anovos.data_analyzer.quality_checker import outlier_detection

output_df, metric_df = outlier_detection(spark, df, detection_side='both', print_impact=True)

This tells us that we have little data for older people, which might be an issue later when we are trying to predict the income of this group, so we should keep this in mind.

We can ignore the value for `education-num`, a categorical column for which the calculations performed by the `outlier_detection` function are meaningless: In its standard configuration, it checks for values that fulfill at least two of the following criteria: They belong to the smallest or largest 5% of values, they deviate from the mean by more than 3 standard deviations, or they lie below `Q1 - 1.5*IQR` or above `Q3 + 1.5*IQR`, where `Q1` and `Q3` are the first and third quartile, respectively, and `IQR` is the Interquartile Range.

**TODO** Add link to docs or code for settings

To better understand this, let's examine the hours worked per week reported by the surveyed people:

In [ ]:
import plotly.express as px

px.histogram(df.toPandas(), x='hours-per-week')

We immediately see that the vast majority of people works a standard 40-hour week, while a significant minority reports anywhere between 20 and 60 hours. However, there are also individuals that work very few hours as well as people that reported almost 100 hours per week, more than twice the median. These are the groups detected by the outlier detector.

We cannot only detect that there are outliers, we can also deal with them right away. For example, let's remove the rows where individuals reported an excessive amount of hours worked per week:

In [ ]:
df, metric_df = outlier_detection(spark, df, detection_side='both', list_of_cols=['hours-per-week'], treatment=True, treatment_method='row_removal')

Let's check that we have indeed reduced the dataset to entries where the number of hours worked per week lies within a common range:

In [ ]:
px.histogram(df.toPandas(), x='hours-per-week')

### Understand how your data is distributed

When familiarizing ourselves with a dataset, one of the first steps is to understand the ranges of values each column contains and how the values are distributed. At the very least, we should learn the minimal and maximal values as well as examine the distribution within that range (e.g., by computing the mean, median, and standard deviation).

This information is vital to know. For many popular kinds of ML models, each feature column should be scaled to the same order of magnitude. Further, ML models will generally only work well for ranges of feature values that they were trained on, so we should check that the data points they see during inference lie within the ranges we find.

_Anovos_' `data_analyzer.stats_generator` module provides an easy way to quickly calculate a set of common properties for the columns in a dataset:

In [ ]:
from anovos.data_analyzer.stats_generator import global_summary
global_summary(spark, df).toPandas()

In most real-world use cases, we are faced with incomplete datasets. For example, customer records might be missing values because we have not had a chance to ask the customer about them. In other cases, there might have been a broken sensor, leading to missing values in a certain time period.

In any case, we should know which columns in our dataset might exhibit such issues in order to consider this during feature selection and modeling. We can use the `data_analyzer.stats_generator` module to check this:

In [ ]:
from anovos.data_analyzer.stats_generator import measures_of_counts
measures_of_counts(spark, df).toPandas()

There are various ways to deal with missing or unknown values. For example, we could replace missing entries in our dataset with the mean or the median of the respective column. If a column contains mostly null values, it might also be an option to drop it entirely. If a dataset contains a lot of unknown values across all of its columns, it will likely be necessary to design a model that can explicitly handle this situation.

Whatever is appropriate in a given scenario, _Anovos_ offers convient functions for this purpose in its `data_transformer` module, which we will have a look at later in this guide.

### Detect correlations within the dataset

In machine learning, we are often interested in predicting a class or value (the _label_) from a set of _features_. To decide which features to use in a specific scenario, it is often helpful to determine which columns in a dataset are correlated with the label column. In other words, we would like to find out which columns hold "predictive power". 

**TODO** Check the terminology

_Anovos_' `data_analyzer.association_evaluator` provides several functions for this purpose. 

A commonly used tool is a correlation matrix. It visualizes the degree of pairwise correlation between multiple columns at once.

In [ ]:
from anovos.data_analyzer.association_evaluator import correlation_matrix
correlation_matrix(spark, df, list_of_cols=['age', 'education-num', 'income']).toPandas()

From the matrix we see that age and education correlate with income: The older or the higher educated a person, the higher the likelihood that they earn above $50k.

**TODO** Add more discussion, point to more advanced functionality

### Examine Drift

Further, we have the drift detection. Drift is a problem for ML models. If over time the distribution changes compared to that of the training, validation, and test data, model performance might degrade.

**TODO** Add a link to a conceptual tutorial on Data Drift (e.g., on Medium)

_Anovos_ provides an entire module dedicated to detecting various kinds of data drift. We recommend you check how the data you're planning to use evolves over time prior to starting feature engineering.

As we only have one dataset, we will artificially create a dataset that has drift by duplicating our dataset and shifting the age column. You could think of this as having surveyed all individuals in the study over their entire life span.

The `drift_detector` helps us ...

We see that ...

# Transform the data

_Key Message(s):_ A basic kind of feature engineering is to apply basic transformations to the data, either to enhance it (e.g., add missing values) or to make it machine-processable (e.g., turn categorical values into cardinal ones)

# Generate a report

_Key Message(s):_ Documenting datasets is an important part of data governance; Anovos provides a set of functions to generate extensive data set reports

## Store the data

Once we've prepared and documented the data, it's time to store it so we can use it to train and evaluate ML models.

Similar to data ingestion, data storage in _Anovos_ is handled through [Spark's versatile capabilities](). Using the `data_ingest.write_dataset` function, we can write our processed DataFrame to disk:

In [ ]:
from anovos.data_ingest.data_ingest import write_dataset
write_dataset(spark, file_path=".", file_type="csv", file_configs={"header": "True", "delimiter": ","})

For now, this final step of data preparation (and this introductory guide) is where _Anovos_' capabilities end. However, over the course of the upcoming releases we will extend _Anovos_ to include adapters for popular AutoML solutions and Feature Stores, allowing you to seamlessly move to model trainin and serving as well as data monitoring.

## What's next?

In this guide, you've had a glimpse at the different capabilities offered by _Anovos_. Of course, we've just scratched the surface and there is much more to see and explore:

- The documentation is a great place to get an overview of the available functionality.
- To see how different parts of _Anovos_ can be used in practice, have a look at the notebooks our team has prepared for each of the modules.
- Finally, to understand how _Anovos_ can be integrated into your Spark ecosystem, see these hints.

**TODO** Add links to the respective resources